In [ ]:
import vsnb # importing automatically registers the ipython magic
import multiprocess as mp

def work(cpus=mp.cpu_count()):
    def f(x):
        return sum(range(x * 100_000_000))
    with mp.Pool(cpus) as p:
        p.map(f, [1] * 2 * cpus)

## Magic: For cells

The `%%monitor` cell magic lets you monitor CPU and memory usage as a cell runs. Just add it at the top of any cell.

In [2]:
%%monitor

work()

Use `--persist` or `-p` if you want the widget to persist after cell execution completes. This can be useful as a reminder of CPU/memory used:

In [3]:
%%monitor -p

work()

Increase update frequency for smoother visualization use four updates per second:

In [28]:
%%monitor -p -i 0.25

work()

In [29]:
%%monitor -p -c

work()

Produce summary percentages instead of gauges (overrides `-p`):

In [6]:
%%monitor -s

work()

CPU %,94,93,92,92,95,95,94,94


Widget fills width of cell by default. To limit width, use -w option:

In [33]:
%%monitor -p -w 100

work()

You can also override the automatic computation of CPU gauges per line:

In [34]:
%%monitor -p -f 4 -w 50

work()

## Context Manager: For scripts or parts of cells

The context manager gives you explicit control over monitoring.
Perfect for Python scripts or when you want programmatic control.

### Basic Context Manager

In [35]:
from vsnb import CPUMonitor

with CPUMonitor(label="Computation Phase", persist=True, color=True):
    work()

All arguments to the `monitor` magic can also be passed to `CPUMonitor`:

In [36]:
with CPUMonitor(summary=True, label="First computation"):
    work()

with CPUMonitor(summary=True, label="Second computation"):
    work()

CPU %,98,97,97,97,98,98,98,99


CPU %,96,96,95,95,96,96,98,96


## Decorator: For functions

Wrap any function with `@monitor` to automatically monitor its execution. The decorator takes the same arguments as the magic and the context manager, but here the label defaults to the function name. 

In [ ]:
from vsnb import monitor

@monitor
def some_function():
    work()

@monitor
def other_function():
    work()      

That lets you track CPU usage for your code in real time:

In [41]:
for _ in range(3):
    some_function()
other_function()

In [42]:
@monitor(persist=True)
def some_function():
    work()

for _ in range(3):
    some_function()

Or profile them using `summary=True`:

In [39]:
@monitor(summary=True)
def some_function():
    work()

@monitor(summary=True)
def other_function():
    work()     

In [13]:
some_function()   
other_function() 

CPU %,95,95,95,95,97,98,98,96


CPU %,94,94,93,94,97,96,96,96
